# Recomendação por Filtragem Colaborativa

Este *notebook* realiza recomendações de ex-candidatos à presidência para usuários, através da filtragem colaborativa.

O *dataset* foi obtido à partir de um formulário criado no *Google Forms*, onde é solicitado do usuário sua avaliação perante aos ex-candidatos, esta avaliação está no intervalo de 1 a 5.

Caso deseje contribuir para o *dataset*, basta preencher o formulário neste [link](https://goo.gl/forms/Cp2glphoDnLQZJof2).

## Importação das Bibliotetcas

In [1]:
from math import *
import numpy as np
import pandas as pd

# Dataset

Inicialmente, precisa-se realizar a leitura do *dataset* e realizar alguns tratamentos de dados.

## Pré-Processamento de Dados

In [4]:
df = pd.read_csv('../dados-pesquisa/avaliacao.csv')

A coluna `Carimbo de data/hora` não será necessário, por isso será removida.

In [6]:
df.drop(['Timestamp'],axis=1,inplace=True)

Os valores presentes no *dataset* possuem o tipo *string*, assim é necessário transformá-los para o tipo *float*.

In [7]:
df = df.astype(float)
df.index = range(1,len(df)+1)

A seguir o *dataset* após estas modificações iniciais.

In [8]:
df.head()

,Geraldo Alckmin,João Amoêdo,Jair Bolsonaro,Guilherme Boulos,Ciro Gomes,Marina Silva,Fernando Haddad,Cabo Daciolo
1,1.0,3.0,1.0,3.0,3.0,3.0,4.0,3.0
2,1.0,3.0,1.0,1.0,3.0,3.0,2.0,1.0
3,2.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0
4,2.0,1.0,1.0,5.0,5.0,4.0,5.0,5.0
5,2.0,5.0,2.0,1.0,2.0,3.0,1.0,1.0


In [9]:
print(f'Atualmente o dataset possui {len(df)} amostras')

Atualmente o dataset possui 106 amostras


# Novo *Dataset* com valores nulos

Como pode ser observado, o *dataset* não possui nenhum dado faltoso, ou seja, todos os candidatos foram avaliados. Para realizar a recomendação, precisa-se ter algum dado faltoso (do tipo `NaN`) e assim o algoritmo recomendar para  o usuário tal candidato.

Para realizar esta operação, criou-se um novo *DataFrame* que receberá o anterior com os dados faltosos, isto é feito com o comando abaixo. Dado o tamanho total do *dataset* (`df.shape`), serão selecionados elementos de forma aleatória que serão "mascarados" para o tipo `NaN`. O total de elementos corresponde à menos de 40% do *dataset*.

**Obs.:** A cada nova execução os valores mudam, justamente pela função selecionar aleatoriamente.

In [10]:
new_df = df.mask(np.random.random(df.shape) < .4)

Para tornar a recomendação mais legível, será criado um novo índice para o *DataFrame* que responderá ao usuário. Assim durante a recomendaçao será passado como paramêtro `eleitor1` ou `eleitor2`, ao invés de apenas um número.

In [11]:
new_df.insert(loc=0, column='eleitor', value= ['eleitor' + str(i) for i in range(1,len(df)+1)])

In [12]:
new_df.index = new_df.eleitor
new_df.drop(['eleitor'],axis=1,inplace=True)

A seguir o novo *dataset* com dados faltosos e índice correspondendo ao eleitor.

In [13]:
new_df.head()

,Geraldo Alckmin,João Amoêdo,Jair Bolsonaro,Guilherme Boulos,Ciro Gomes,Marina Silva,Fernando Haddad,Cabo Daciolo
eleitor,,,,,,,,
eleitor1,1.0,3.0,1.0,NaN,3.0,3.0,4.0,NaN
eleitor2,1.0,NaN,NaN,NaN,NaN,3.0,2.0,NaN
eleitor3,2.0,2.0,1.0,NaN,NaN,3.0,NaN,2.0
eleitor4,2.0,NaN,1.0,NaN,5.0,4.0,5.0,NaN
eleitor5,NaN,5.0,NaN,NaN,2.0,3.0,NaN,1.0


# Implementação da Filtragem Colaborativa

**Obs.:** Para todas as seguintes funções, precisou-se realizar uma alteração do algoritmo padrão. Diferente de um dicionário, cada eleitor no *DataFrame* possui a coluna do candidato, mesmo que não tenha valor para ele, assim nas condições do `if` são verificados se o valor da coluna é `isnan` ou `not isnan`.

## Distância de *Minkowski*

In [14]:
def minkowski(rating1, rating2, r = 1):
    distance = 0
    commonRatings = False
    for candidate in rating1.keys():
        
        if not isnan(rating1[candidate]) and not isnan(rating2[candidate]):
            
            distance += pow(abs(rating1[candidate] - rating2[candidate]), r)
            commonRatings = True
    
    if commonRatings:
        return pow(distance, 1/r)
    else:
        return float('Inf')

Exemplo de utilização da função.

In [15]:
print(minkowski(new_df.loc["eleitor1"], new_df.loc["eleitor2"]))

2.0


Para facilitar as próximas funções, criou-se uma lista que possui os eleitores, ou seja, o índice do *DataFrame*.

In [16]:
voters = ['eleitor' + str(i) for i in range(1,len(df)+1)]

# Vizinhos mais próximos

In [17]:
def neighbor(votername):
    distances = []
    for voter in voters:
        if voter != votername:
            distance = minkowski(new_df.loc[voter], new_df.loc[votername])

            distances.append((distance, voter))
            
    distances.sort()
    return distances

Exemplo de utilização da função.

In [18]:
print(neighbor("eleitor1"))

[(0.0, 'eleitor102'), (0.0, 'eleitor11'), (0.0, 'eleitor21'), (0.0, 'eleitor31'), (0.0, 'eleitor95'), (1.0, 'eleitor101'), (1.0, 'eleitor17'), (1.0, 'eleitor32'), (1.0, 'eleitor48'), (1.0, 'eleitor54'), (1.0, 'eleitor85'), (2.0, 'eleitor103'), (2.0, 'eleitor2'), (2.0, 'eleitor22'), (2.0, 'eleitor3'), (2.0, 'eleitor30'), (2.0, 'eleitor36'), (2.0, 'eleitor40'), (2.0, 'eleitor59'), (2.0, 'eleitor63'), (2.0, 'eleitor65'), (2.0, 'eleitor66'), (2.0, 'eleitor67'), (2.0, 'eleitor83'), (3.0, 'eleitor105'), (3.0, 'eleitor39'), (3.0, 'eleitor42'), (3.0, 'eleitor44'), (3.0, 'eleitor45'), (3.0, 'eleitor5'), (3.0, 'eleitor56'), (3.0, 'eleitor75'), (3.0, 'eleitor77'), (3.0, 'eleitor80'), (3.0, 'eleitor86'), (3.0, 'eleitor89'), (3.0, 'eleitor93'), (3.0, 'eleitor94'), (4.0, 'eleitor104'), (4.0, 'eleitor13'), (4.0, 'eleitor14'), (4.0, 'eleitor16'), (4.0, 'eleitor18'), (4.0, 'eleitor20'), (4.0, 'eleitor38'), (4.0, 'eleitor46'), (4.0, 'eleitor50'), (4.0, 'eleitor57'), (4.0, 'eleitor58'), (4.0, 'eleitor6')

## Recomendação

Pode acontecer de a lista `recommendations` retornar nenhum valor, isto pode acontecer caso o eleitor e seu vizinho mais próximo possui apenas um valor válido e o resto `NaN` por exemplo. Para evitar este resultado, adicionou-se uma nova condição que caso `len(recommendations) == 0`, o algoritmo será chamado recursivamente e tentará realizar a recomendação com o próximo vizinho mais próximo.

In [21]:
def recommend(votername, nearestNeighbor = 0):
    i = nearestNeighbor
    
    nearest = neighbor(votername)[i][1]
    
    recommendations = []
    
    neighborRatings = new_df.loc[nearest]
    voterRatings = new_df.loc[votername]
    
    for candidate in neighborRatings.keys():
        if isnan(voterRatings[candidate]) and not isnan(neighborRatings[candidate]):
            
            recommendations.append((candidate, neighborRatings[candidate]))
    
    while len(recommendations) == 0:
        i+=1
        recommendations = recommend(votername,i)
    
    return sorted(recommendations,
        key=lambda candidateTuple: candidateTuple[1],
        reverse = True)

Exemplo de utilização da função.

In [22]:
print(recommend("eleitor1"))

[('Guilherme Boulos', 4.0), ('Cabo Daciolo', 1.0)]


In [23]:
print(recommend("eleitor2"))

[('João Amoêdo', 3.0), ('Jair Bolsonaro', 1.0)]


In [24]:
print(recommend("eleitor3"))

[('Ciro Gomes', 2.0), ('Fernando Haddad', 1.0)]


In [25]:
print(recommend("eleitor4"))

[('Guilherme Boulos', 3.0), ('João Amoêdo', 2.0)]


In [26]:
print(recommend("eleitor5"))

[('Fernando Haddad', 3.0), ('Jair Bolsonaro', 1.0)]


In [27]:
new_df.head()

,Geraldo Alckmin,João Amoêdo,Jair Bolsonaro,Guilherme Boulos,Ciro Gomes,Marina Silva,Fernando Haddad,Cabo Daciolo
eleitor,,,,,,,,
eleitor1,1.0,3.0,1.0,NaN,3.0,3.0,4.0,NaN
eleitor2,1.0,NaN,NaN,NaN,NaN,3.0,2.0,NaN
eleitor3,2.0,2.0,1.0,NaN,NaN,3.0,NaN,2.0
eleitor4,2.0,NaN,1.0,NaN,5.0,4.0,5.0,NaN
eleitor5,NaN,5.0,NaN,NaN,2.0,3.0,NaN,1.0


In [41]:
df.head()

,Geraldo Alckmin,João Amoêdo,Jair Bolsonaro,Guilherme Boulos,Ciro Gomes,Marina Silva,Fernando Haddad,Cabo Daciolo
1,1.0,3.0,1.0,3.0,3.0,3.0,4.0,3.0
2,1.0,3.0,1.0,1.0,3.0,3.0,2.0,1.0
3,2.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0
4,2.0,1.0,1.0,5.0,5.0,4.0,5.0,5.0
5,2.0,5.0,2.0,1.0,2.0,3.0,1.0,1.0
